# Serving a Keras Resnet Model

See `resnet_training_to_serving.ipynb` for the Tensorflow Estimator API version and an outline of steps.

The step to creating a servable Resnet50 model from Keras is simpler as Keras already contains in its applications package, a pre-trained Resnet50 model using imagenet. We simply have to take the Keras model, and set its inputs field and output fields in the prediction signature definition in the Tensorflow saved_model library. (Compare this to the Estimator API, where the input signature is defined by the `serving_input_receiver_fn()` dictionary argument, and the output signature is defined by the EstimatorSpec constructor's export_outputs field in the model_fn.

See https://github.com/keras-team/keras/blob/master/keras/applications/resnet50.py for the implementation of ResNet50.

# Preamble

Import the required libraries.

In [0]:
# Import Keras libraries
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras import backend as K
import numpy as np

# Import Tensorflow saved model libraries
import tensorflow as tf
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

In [0]:
# Constants

_DEFAULT_IMAGE_SIZE = 224
TOP_K = 5

# Download the Keras model

Creating a ResNet50 model will automatically download a saved model if one does not already exist locally.


# Setting the Output Directory

Unlike the Estimator API that automatically creates a servable version number using the unix timestamp, building a servable model directly from a tensorflow graph requires creating an explicit integer version number.

Note that if you've successfully saved the servable in a directory, trying to save another servable will fail hard. You always want to increment your version number, or otherwise delete the output directory and re-run the servable creation code.

In [0]:
VERSION_NUMBER=1
SERVING_DIR="keras_resnet_servable/" + str(VERSION_NUMBER)

# Create the Servable Model

Keras has a prepackaged ImageNet-trained ResNet50 model which takes in a 4d input tensor and outputs a list of class probabilities for all of the classes.

We will create a servable model whose input and output formats are identical to that provided in the Estimator API version (`resnet_training_to_serving_solution.ipynb`). Basically, the input needs is a list of jpegs, and the output needs to contain the top k classes and probabilities. This will require connecting a Tensorflow placeholder and preprocessing functions to the Keras model input.

In [0]:
# Preprocessing helper function similar to `resnet_training_to_serving_solution.ipynb`.

def preprocess_image(encoded_image, height=_DEFAULT_IMAGE_SIZE, width=_DEFAULT_IMAGE_SIZE):
  """Preprocesses the image by subtracting out the mean from all channels.
  Args:
    image: A jpeg-formatted byte stream represented as a string.
  Returns:
    A 3d tensor of image pixels normalized using the resnet50-specific Keras default preprocessor
  """
  image = tf.image.decode_jpeg(encoded_image, channels=3)
  image = tf.to_float(image)
  image = preprocess_input(image)
  return image

# Input to the Keras Model

As with the Estimator, the graph starts at the bottom with a [Tensorflow placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder), which can receive an array of jpeg-encoded strings
from the client. The placeholder needs to be further processed before being fed into the Keras model.

**Exercise:** You will need to modify the input to the Keras Model to be compliant with `client/resnet_client.py`.
As in the Estimator example, you need to preprocess a list of jpegs encoded as strings into a 4D tensor `processed_images` with appropriate dimensions [-1, 224, 224, 3]. (-1 means undetermined.)

**HINT:** See resnet_model_fn() in `resnet_training_to_serving_solution.ipynb`.

In [0]:
images = tf.placeholder(dtype=tf.string, shape=[None])

processed_images = tf.map_fn(preprocess_image, images, dtype=tf.float32)  # Convert to a list of tensors
processed_images = tf.stack(processed_images)  # Convert list of tensors to tensor of tensors
processed_images = tf.reshape(tensor=processed_images,  # Reshape to ensure TF graph knows the final dimensions
                            shape=[-1, _DEFAULT_IMAGE_SIZE, _DEFAULT_IMAGE_SIZE, 3])

# Load the Keras Model

The Keras Model uses Tensorflow as its backend, and therefore its inputs and outputs can be treated as elements of a Tensorflow graph. In other words, you can provide an input that is a Tensorflow tensor, and read the model output like a Tensorflow tensor!

**Exercise**: Assign the 4D tensor of processed images to the input_tensor parameter.

In [0]:
# Load the ResNet50 Keras Model with processed_images as input
model = ResNet50(include_top=True, weights='imagenet', input_tensor=processed_images)
# Rename the model to 'resnet'
model.name = 'resnet'

# Output from the Keras Model

**Exercise:** Postprocess the output of the Keras Model to return the top k classes and probabilities predicted by the model. The Keras ResNet50 model returns an array of 1001 probabilities for each of the classes + the unknown class.

**HINT:** This is similar to but slightly different from resnet_model_fn() in `resnet_training_to_serving_solution.ipynb`.

In [0]:
top_k_probs, top_k_classes = tf.nn.top_k(model.output, k=TOP_K)

predictions = {
  'classes': top_k_classes,
  'probabilities': top_k_probs)
}

# Create a saved model builder, and save the model
builder = saved_model_builder.SavedModelBuilder(SERVING_DIR)

signature = predict_signature_def(inputs={'images': images},
                                  outputs=predictions)



# Build the Servable

In [0]:
with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()